In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pickle
import os
import base64
from os import listdir
import csv
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


In [4]:
file_path = '/content/drive/My Drive/final_year/profile_dict.pickle'


with open(file_path, 'rb') as f:
    encoded_data = f.read()

# Decode the Base64-encoded data
decoded_bytes = base64.b64decode(encoded_data)

# Deserialize the object using pickle
img_dict = pickle.loads(decoded_bytes)
print(img_dict.keys())


dict_keys(['x', 'y'])


In [5]:
print(len(img_dict['y']),len(img_dict['x']))

925 925


In [6]:
import numpy as np
from scipy.ndimage import zoom

x_resized = []
y_resized = []

for i in img_dict['x']:
    resized_x = zoom(i, (208/i.shape[0], 208/i.shape[1], 1), order=1)
    x_resized.append(resized_x)

for i in img_dict['y']:

    y_resized.append(i)

# Convert lists to numpy arrays
x_resized = np.array(x_resized)
y_resized = np.array(y_resized)

print("Resized x shape:", x_resized.shape)
print("Resized y shape:", y_resized.shape)


Resized x shape: (925, 208, 208, 3)
Resized y shape: (925, 5)


In [7]:
x_data=[]
y_data=[]
y_resized=y_resized.astype(float)
y_resized.shape
x_resized.shape
print(y_resized[0])
for i in range(len(y_resized)):
    for j in range(5):
        y_data.append(y_resized[i])
len(y_data)

[0.17 0.2  0.19 0.2  0.23]


4625

In [8]:

for img in x_resized:
    img_blur = cv2.GaussianBlur(img, (5, 5), 0)
    img_blur= cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY)
    img_blur=np.expand_dims(img_blur, axis=-1)
    x_data.append(img_blur)

    edges = cv2.Canny(img, 100, 200)
    edges=np.expand_dims(edges, axis=-1)
    x_data.append(edges)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    thresh=np.expand_dims(thresh, axis=-1)
    x_data.append(thresh)

    datagen = ImageDataGenerator(
    rotation_range=20,
    #     width_shift_range=0.2,
    #     height_shift_range=0.2,
    #     shear_range=0.2,
    #     zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


    img = img.reshape((1,) + img.shape)


    augmented_images = []
    num_augmented_images = 2  # Number of augmented images to generate

    for _ in range(num_augmented_images):
        for batch in datagen.flow(img, batch_size=1):
            augmented_images.append(batch[0].astype('uint8'))
            break
    for i in augmented_images:
        i = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
        i=np.expand_dims(i,axis=-1)

        x_data.append(i)








In [9]:
for i,x in enumerate(x_data):
    print(x.shape)
    print(y_data[i])
    plt.imshow(x,cmap='gray')
    plt.show()
    if i==20:
        break

Output hidden; open in https://colab.research.google.com to view.

In [10]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Dropout,MaxPooling2D,Dense,Input,Reshape, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
import math
from tensorflow.keras.optimizers import Adam

In [11]:
x_data=np.array(x_data)
y_data=np.array(y_data)
print(x_data.shape,"   ",y_data.shape)

(4625, 208, 208, 1)     (4625, 5)


In [12]:
input_cla=Input(shape=(208,208,1))
fm_size=16
x=Conv2D(fm_size,(3,3),activation='relu',padding='same')(input_cla)
x = BatchNormalization()(x)
x=MaxPooling2D((2,2),padding='same')(x)

fm_size=fm_size*2
x=Conv2D(fm_size,(3,3),activation='relu',padding='same')(x)
x = BatchNormalization()(x)
x=MaxPooling2D((2,2),padding='same')(x)

fm_size=fm_size*2
x=Conv2D(fm_size,(3,3),activation='relu',padding='same')(x)
x = BatchNormalization()(x)
x=MaxPooling2D((2,2),padding='same')(x)

fm_size=fm_size*2
x=Conv2D(fm_size,(3,3),activation='relu',padding='same')(x)
x = BatchNormalization()(x)
x=MaxPooling2D((2,2),padding='same')(x)

fm_size=fm_size*2
x=Conv2D(fm_size,(3,3),activation='relu',padding='same')(x)
x = BatchNormalization()(x)
x=MaxPooling2D((2,2),padding='same')(x)

fm_size=fm_size*2
x=Conv2D(fm_size,(3,3),activation='relu',padding='same')(x)
x = BatchNormalization()(x)
x=MaxPooling2D((2,2),padding='same')(x)

fm_size=fm_size*2
x=Conv2D(fm_size,(3,3),activation='relu',padding='same')(x)
x = BatchNormalization()(x)
x=MaxPooling2D((2,2),padding='same')(x)

fm_size=fm_size*2
x=Conv2D(fm_size,(3,3),activation='relu',padding='same')(x)
x = BatchNormalization()(x)
x=MaxPooling2D((2,2),padding='same')(x)

em_size=math.ceil(208/(2**8))
x = Reshape((em_size * em_size * fm_size,))(x)
dropout_rate=0.5
x=Dense(128,activation='relu',input_shape=(em_size*em_size*fm_size,))(x)
x = Dropout(dropout_rate)(x)
x=Dense(64,activation='relu')(x)
x = Dropout(dropout_rate)(x)
x=Dense(32,activation='relu')(x)
x = Dropout(dropout_rate)(x)
x=Dense(16,activation='relu')(x)
x = Dropout(dropout_rate)(x)
output=Dense(5)(x)

classifier=Model(input_cla,output)
adam = Adam(learning_rate=1e-4)
# adam=Adam
loss = MeanSquaredError()
classifier.compile(optimizer=adam, loss=loss, metrics = [tf.keras.metrics.MeanSquaredError()])

classifier.summary()




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 208, 208, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 208, 208, 16)      160       
                                                                 
 batch_normalization (Batch  (None, 208, 208, 16)      64        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 104, 104, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 104, 104, 32)      4640      
                                                                 
 batch_normalization_1 (Bat  (None, 104, 104, 32)      128   

In [18]:
from tensorflow.keras.callbacks import ReduceLROnPlateau


# Define the ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(factor=0.3, patience=5, verbose=1)

# Train the model with the callback
history = classifier.fit(x_data, y_data, epochs=20, batch_size=64, callbacks=[reduce_lr], validation_split=0.2)


Epoch 1/20
58/58 [==============================] - 6s 111ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0013 - val_mean_squared_error: 0.0013 - lr: 2.7000e-06
Epoch 2/20
58/58 [==============================] - 6s 108ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0013 - val_mean_squared_error: 0.0013 - lr: 2.7000e-06
Epoch 3/20
58/58 [==============================] - 6s 103ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0013 - val_mean_squared_error: 0.0013 - lr: 2.7000e-06
Epoch 4/20
58/58 [==============================] - 6s 109ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0013 - val_mean_squared_error: 0.0013 - lr: 2.7000e-06
Epoch 5/20
58/58 [==============================] - 6s 103ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0013 - val_mean_squared_error: 0.0013 - lr: 2.7000e-06
Epoch 6/20
58/58 [==============================] - ETA: 0s - loss: 0.0013 - mean_squared_error: 0.0013
E

In [20]:
# Save model architecture, weights, and optimizer state
classifier.save('/content/drive/My Drive/final_year/profile_model_synthetic.keras')

# Save training history
np.save('/content/drive/My Drive/final_year/profile_history_synthetic.npy', history.history)


In [21]:
import cv2
import numpy as np

x_gray = []
for img in x_resized:
    img_arr_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_arr_gray_expanded = np.expand_dims(img_arr_gray, axis=-1)
    x_gray.append(img_arr_gray_expanded)
x_gray = np.array(x_gray)
x_gray.shape


(925, 208, 208, 1)

In [22]:
x_train=x_gray
y_train=y_resized

In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

val_preds=classifier.predict(x_train)

mse=history.history['loss'][-1]
val_mse=history.history['val_loss'][-1]

mae=mean_absolute_error(y_train,val_preds)

rmse=np.sqrt(val_mse)
r2 = r2_score(y_train, val_preds)

# Print the calculated metrics
print("Mean Squared Error (MSE):", mse)
print("Validation Mean Squared Error (val_MSE):", val_mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R^2 score:", r2)

29/29 [==============================] - 2s 14ms/step
Mean Squared Error (MSE): 0.0013125365367159247
Validation Mean Squared Error (val_MSE): 0.0013018922181800008
Mean Absolute Error (MAE): 0.024433428146516955
Root Mean Squared Error (RMSE): 0.03608174355792692
R^2 score: -0.191922265739035
